In [ ]:
import pandas as pd
import numpy as np
import plotly_express as px
import hashlib

# Load datasets
data_athletes = pd.read_csv('Data/athlete_events.csv')
data_noc = pd.read_csv('Data/noc_regions.csv')

### Which countries are included in the dataset?

In [28]:
# Extract unique region names
unique_regions = data_noc['region'].unique()
exclusions_region = ['Individual Olympic Athletes', 'NA'] # Exclusion list for regions

# TUV was described as 'NA' in the region column but had a name in the notes column
if 'Tuvalu' not in unique_regions:
    # Manually add 'Tuvalu' to the list of unique regions
    unique_regions = list(unique_regions) + ['Tuvalu']

# Print the list of unique region names
for region in unique_regions:
    if region not in exclusions_region:
        print(region)

Afghanistan
Curacao
Albania
Algeria
Andorra
Angola
Antigua
Australia
Argentina
Armenia
Aruba
American Samoa
Austria
Azerbaijan
Bahamas
Bangladesh
Barbados
Burundi
Belgium
Benin
Bermuda
Bhutan
Bosnia and Herzegovina
Belize
Belarus
Czech Republic
Boliva
Botswana
Brazil
Bahrain
Brunei
Bulgaria
Burkina Faso
Central African Republic
Cambodia
Canada
Cayman Islands
Republic of Congo
Chad
Chile
China
Ivory Coast
Cameroon
Democratic Republic of the Congo
Cook Islands
Colombia
Comoros
Cape Verde
Costa Rica
Croatia
Greece
Cuba
Cyprus
Denmark
Djibouti
Dominica
Dominican Republic
Ecuador
Egypt
Eritrea
El Salvador
Spain
Estonia
Ethiopia
Russia
Fiji
Finland
France
Germany
Micronesia
Gabon
Gambia
UK
Guinea-Bissau
Georgia
Equatorial Guinea
Ghana
Grenada
Guatemala
Guinea
Guam
Guyana
Haiti
Honduras
Hungary
Indonesia
India
Iran
Ireland
Iraq
Iceland
Israel
Virgin Islands, US
Italy
Virgin Islands, British
Jamaica
Jordan
Japan
Kazakhstan
Kenya
Kyrgyzstan
Kiribati
South Korea
Kosovo
Saudi Arabia
Kuwait
Laos
L

### How many countries are included in the dataset?

In [29]:
# Filter out excluded regions and count
filtered_regions = [
    region for region in unique_regions
        if region not in exclusions_region]

number_of_unique_regions = len(filtered_regions)

print(f"Total number of unique regions: {number_of_unique_regions}")

Total number of unique regions: 207


### Which sports are in the event?

In [ ]:
sports_list = data_athletes['Sport'].unique()

print("Types of sports:")
for sport in sports_list:
    print(sport)

### What types of medals are won?

In [31]:
medal_types = data_athletes['Medal'].unique()

# Removes the rows where no medals has been won
medal_types_filtered = np.delete(medal_types, 0)

print("Types of medals:")
for medal in medal_types_filtered:
    print(medal)

Types of medals:
Gold
Bronze
Silver


### Sex distribiution chart

In [32]:
# Remove duplicate entries based on 'ID' to ensure each athlete is counted only once.
data_unique_athletes = data_athletes.drop_duplicates(subset=['ID'])

# Count the number of male and female participants
sex_distribution = data_unique_athletes['Sex'].value_counts()

# Creating a pie chart
sex_distribution_piechart = px.pie(sex_distribution, 
            # The lambda function in the names argument is used to map "F" to "Female" and "M" to "Male".
             names=sex_distribution.index.map(lambda x: 'Female' if x == 'F' else 'Male'),
             values=sex_distribution.values,
             title='Sex distribution of all athletes',
             labels={'names' : 'Sex', "values" : 'Amount'})

sex_distribution_piechart.write_html(("../Projekt_OS_Australien/Visualisering/Sex_distribution_piechart.html"))

### Top 10 countries based on total medals won

In [33]:
# Creates a new dataframe grouping that only shows the columns "Team" and "Total Medals"
# Value_counts Counts the values for each row
# Unstack transforms the groupby into a new dataframe
# fillna changes the missing data to having a value of 0 so it wont effect the counting
# Sum counts everthing in the first axis which is "Total Medals" after we used reset_index to both reset the index and change the name of the second column.
country_medals = data_athletes.groupby("NOC")["Medal"].value_counts().unstack().fillna(0).sum(axis=1).reset_index(name="Total Medals")

top_ten_countries = country_medals.sort_values(by="Total Medals", ascending=False).head(10)

top_ten_countries_diagram = px.bar(
    top_ten_countries,
    x="NOC", y="Total Medals",
    title="Top 10 countries based on total medals won:",
    color="NOC",
    labels={"NOC" : "Countries", "Total Medals" : "Medals"}
)

top_ten_countries_diagram.write_html("../Projekt_OS_Australien/Visualisering/Top_ten_countries_medals.html")

### Age Statistics

In [ ]:
# Create dataframe grouping with athletes age, sex and medal
# Dropping rows without age data and filling NaN in Medals with "No medal"
# Showing age statistics overall and by sex
athlete_age_medal = data_athletes[["Sex", "Age", "Medal"]].dropna(subset=["Age"]).fillna({"Medal": "No medal"})

age_stats = athlete_age_medal["Age"]
age_stats_by_sex = athlete_age_medal.groupby("Sex")["Age"]

In [ ]:
age_stats.describe()

In [ ]:
age_stats_by_sex.describe()

### Medals by age group

In [ ]:
# Defining age bins
# Creating new column "Age Group" based on the age bins
# Group by "Sex", "Age Group" and "Medals" and count the medal occurrences
# Sort the columns, gold first, no medal last.
age_bins = [10, 20, 30, 40, 50, 60, 70, 80, 90]
athlete_age_medal["Age Group"] = pd.cut(athlete_age_medal["Age"], bins=age_bins, labels=[f'{i}-{i+9}' for i in age_bins[:-1]])
age_group_medals = athlete_age_medal.groupby(["Sex", "Age Group", "Medal"], observed=False).size().unstack(fill_value=0)
age_group_medals = age_group_medals[["Gold", "Silver", "Bronze"]]

In [ ]:
age_group_medals

In [39]:
# Melt the DataFrame for easier plotting
# Create bar chart using Plotly Express
age_group_medals_melted = pd.melt(age_group_medals.reset_index(), id_vars=["Sex", "Age Group"], var_name="Medal", value_name="Count")

age_group_medals_diagram = px.bar(
    age_group_medals_melted,
    x="Age Group",
    y="Count",
    color="Medal",
    color_discrete_map={"Gold": "gold", "Silver": "silver", "Bronze": "brown"},
    barmode="group",
    facet_col="Sex",
    category_orders={"Medal": ["Gold", "Silver", "Bronze", "No medal"]},
    labels={"Count": "Medal Count", "Age Group": "Age Group"},
    title="Medal Counts per Age Group and Sex"
)

age_group_medals_diagram.write_html("../Projekt_OS_Australien/Visualisering/Age_group_medals_by_sex.html")

### Anonymization function for names in dataset

In [ ]:
#Anonymize a name using SHA-256 hashing algorithm
def anonymize_names(name):
    name_bytes = name.encode() # Transforming the string to bytes with .encode(), since the hash function demands it 
    hash_object = hashlib.sha256() # Creating an SHA-256 hashobject with hashlib.sha256()
    hash_object.update(name_bytes) # Updating the hashobject with the koded name-string (name_bytes)
    hashed_name = hash_object.hexdigest() # Making the hashe object into hexdecimal format
    return hashed_name

# Using the function to anonymize the name column
data_athletes['Name'] = data_athletes['Name'].apply(anonymize_names)

### Number of Medals per Sport for Australia

In [ ]:
# Filter for Australian participants, included Australasia since Australia was a part of that subregion.
australian_athletes = data_athletes[data_athletes['NOC'].isin(['AUS', 'ANZ'])]

# Filter for rows where a medal has been won
australian_medals = australian_athletes.dropna(subset=['Medal'])

# Group by sport and count the number of medals
medal_counts = australian_medals['Sport'].value_counts()

# Create a bar chart to visualize the results
medals_per_sport_australia = px.bar(medal_counts, 
             x=medal_counts.index, 
             y=medal_counts.values, 
             title='Number of Medals per Sport for Australia',
             labels={'x': 'Sport', 'y': 'Number of Medals'})

# Rotating the values under the x-axis
medals_per_sport_australia.update_layout(xaxis_tickangle=-90)

medals_per_sport_australia.write_html("../Projekt_OS_Australien/Visualisering/Medals_per_sport_australia.html")

### Amount of medals per olympic games for Australia

In [46]:
# Create dataframe grouping australian medals per season, year and medal amounts
# Sort the columns, gold first, bronze last
# Create stacked bar chart to show results

australian_medals_per_year = australian_medals.groupby(["Season", "Year", "Medal"]).size().unstack(fill_value=0)
australian_medals_per_year = australian_medals_per_year[["Gold", "Silver", "Bronze"]].reset_index()


australian_medals_per_year_diagram = px.bar(
    australian_medals_per_year,
    x="Year",
    y=["Gold", "Silver", "Bronze"],
    color_discrete_map={"Gold": "gold", "Silver": "silver", "Bronze": "brown"},
    barmode="stack",
    facet_col="Season",
    labels={"value": "Medal Count", "Year": "Year"},
    title="Amount of medals per olympic games for Australia",
)

australian_medals_per_year_diagram.write_html("../Projekt_OS_Australien/Visualisering/Medals_per_year_australia.html")

### Histogram over the ages of Australian athletes

In [ ]:
# Creating the diagram
Histogram_Australia = px.histogram(
    australian_athletes,
    x="Age",
    nbins=25,
    title="Age of Australian olympic athletes"
)

# Creating a gap between each bar so it's easier to look at
Histogram_Australia.update_layout(
    bargap=0.2
)

Histogram_Australia.write_html("../Projekt_OS_Australien/Visualisering/Histogram_Australia_Ages.html")

## Swimming 

### Medals per country competing in Swimming

In [ ]:
filter_sport = "Swimming"

# Filteres the data so only swimmers are left
filitered_swimming = data_athletes[data_athletes["Sport"] == filter_sport]

# Creates a series by grouping NOC and medal column together.
# .size() counts the values in each group
# unstack(fill_value=0) reshapes it into a dataframe again, while also changing the NaN values to 0
# reset_index() resets the index again
medals_swimming = filitered_swimming.groupby(["NOC", "Medal"]).size().unstack(fill_value=0).reset_index()

# Filtering out the countries who has participated but has not won any medals
medals_swimming = medals_swimming[medals_swimming[["Gold", "Silver", "Bronze"]].sum(axis=1) > 0]

# Sorting the countries based on total medals for easier readability
medals_swimming["Total Medals"] = medals_swimming[["Gold", "Silver", "Bronze"]].sum(axis=1)
medals_swimming = medals_swimming.sort_values(by="Total Medals", ascending=False)

# Taking the top 20 countries for easier readability
medals_swimming = medals_swimming.head(20)

medals_country_swimming = px.bar(
    medals_swimming,
    x="NOC",
    y=["Gold", "Silver", "Bronze"],
    labels={"NOC": "Country", "value" : "Medals"},
    barmode="group",
    title="Amount of Swimming medals per country"
)

medals_country_swimming.write_html("../Projekt_OS_Australien/Visualisering/Medals_country_swimming.html")


### Age groups for swimming

In [ ]:
age_distrobution_swimming = px.histogram(
    filitered_swimming,
    x="Age",
    nbins=10,
    title="Age distrubution of swimmers"
)

age_distrobution_swimming.update_layout(
    bargap=0.1
)

age_distrobution_swimming.write_html("../Projekt_OS_Australien/Visualisering/Age_distrubution_swimming.html")

<span style="color:red"><b><<<<<<< local</b></span>

### Tug-Of-War

## Medal distribution among countries in Tug-Of-War

In [ ]:
# Filter for Tug-Of-War
tug_of_war_data = data_athletes[data_athletes['Sport'] == 'Tug-Of-War']

# Plot for medal distribution among countries in Tug-Of-War
tug_of_war_medals = tug_of_war_data[tug_of_war_data['Medal'].notna()]
medal_distribution_tug_of_war = tug_of_war_medals['NOC'].value_counts().reset_index()
medal_distribution_tug_of_war.columns = ['NOC', 'Medals']

medal_distribution_tug_of_war = px.bar(
    medal_distribution_tug_of_war.head(10),
    x='NOC', 
    y='Medals',
    title='Top 10 Countries in Medal Count for Tug-Of-War',
    labels={
        'NOC': 'National Olympic Committee',
        'Medals': 'Number of Medals'
    }
)

medal_distribution_tug_of_war.write_html("../Projekt_OS_Australien/Visualisering/Medal_distribution_tug_of_war.html")

<span style="color:red"><b>=======</b></span>

## Cross country skiing

### Cross country skiing medals per country

In [73]:
# Making new dataframe only containing cross country skiers
# Grouping by NOC and amount of medals
cross_country_skiers = data_athletes[data_athletes["Sport"] == "Cross Country Skiing"]
cross_country_medals = cross_country_skiers.groupby(["NOC", "Medal"]).size().unstack(fill_value=0).reset_index()

# Sorting each NOC by total medals for nicer looking diagram
cross_country_medals["Total Medals"] = cross_country_medals["Gold"] + cross_country_medals["Silver"] + cross_country_medals["Bronze"]
cross_country_medals_sorted = cross_country_medals.sort_values(by='Total Medals', ascending=False)
cross_country_medals_sorted[["NOC", "Gold", "Silver", "Bronze"]]

Medal,NOC,Gold,Silver,Bronze
12,NOR,54,73,37
6,FIN,35,42,64
17,SWE,54,30,39
19,URS,44,35,32
10,ITA,15,21,25
14,RUS,23,14,12
9,GER,6,19,10
16,SUI,3,0,10
3,CZE,1,5,6
5,EUN,6,2,4


In [76]:
# Creating stacked bar diagram showing amount of cross country skiing medals for each country
cross_country_medals_per_country_diagram = px.bar(
   cross_country_medals_sorted,
    x="NOC",
    y=["Gold", "Silver", "Bronze"],
    color_discrete_map={"Gold": "gold", "Silver": "silver", "Bronze": "brown"},
    barmode="stack",
    labels={"value": "Medal Count", "NOC": "Country"},
    title="Amount of cross country skiing medals per country",
)

cross_country_medals_per_country_diagram.write_html("../Projekt_OS_Australien/Visualisering/Cross_country_medals_per_country.html")

### Cross country skiing age groups

In [89]:
# Creating histogram showing cross country skiers ages
cross_country_skiers_age_diagram = px.histogram(
    cross_country_skiers,
    x="Age"
)
cross_country_skiers_age_diagram.update_layout(
    bargap=0.2
)
cross_country_skiers_age_diagram.write_html("../Projekt_OS_Australien/Visualisering/Cross_country_skiers_age.html")

<span style="color:red"><b>>>>>>>> remote</b></span>